In [65]:
import sys
!{sys.executable} -m pip install datasets --upgrade

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [27]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Prevent truncation of column width
pd.set_option('display.width', None)

In [14]:
import anthropic
import os
import requests

# Get the API URL and key from environment variables
API_KEY = os.environ.get('ANTHROPIC_API_KEY')

if API_KEY is None:
    raise EnvironmentError("API_URL and API_KEY must be set in the .env file.")

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=API_KEY,
)
message = client.messages.create(
    model="claude-2.1",
    max_tokens=1000,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "how current is your information?"
                }
            ]
        }
    ]
)
print(message.content)

[ContentBlock(text="I'm afraid I don't have enough context to state definitively how current my information is. As an AI assistant created by Anthropic to be helpful, harmless, and honest, I respond based on my training. I don't have an independent measure of the recency of my knowledge. I'd be happy to provide additional clarification if you have a more specific question.", type='text')]


One really nice feature of [Claude.ai](https://www.claude.ai) is the ability to upload PDFs. Let's mock up that feature in a notebook, and then test it out by summarizing a long PDF.

In [16]:
!curl -O https://arxiv.org/pdf/2212.08073.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2039k  100 2039k    0     0  3830k      0 --:--:-- --:--:-- --:--:-- 3825kk    0     0  2047k      0 --:--:-- --:--:-- --:--:-- 2046k


Now, we'll use pypdf to read the pdf. It's not identical to what Claude.ai uses behind the scenes, but it's pretty close.

In [19]:
from pypdf import PdfReader

reader = PdfReader("2212.08073.pdf")
number_of_pages = len(reader.pages)
text = ''.join([page.extract_text() for page in reader.pages])
print(text[:2155])

Constitutional AI: Harmlessness from AI Feedback
Yuntao Bai∗, Saurav Kadavath, Sandipan Kundu, Amanda Askell, Jackson Kernion,
Andy Jones, Anna Chen, Anna Goldie, Azalia Mirhoseini, Cameron McKinnon,
Carol Chen, Catherine Olsson, Christopher Olah, Danny Hernandez, Dawn Drain,
Deep Ganguli, Dustin Li, Eli Tran-Johnson, Ethan Perez, Jamie Kerr, Jared Mueller,
Jeffrey Ladish, Joshua Landau, Kamal Ndousse, Kamile Lukosuite, Liane Lovitt,
Michael Sellitto, Nelson Elhage, Nicholas Schiefer, Noemi Mercado, Nova DasSarma,
Robert Lasenby, Robin Larson, Sam Ringer, Scott Johnston, Shauna Kravec,
Sheer El Showk, Stanislav Fort, Tamera Lanham, Timothy Telleen-Lawton, Tom Conerly,
Tom Henighan, Tristan Hume, Samuel R. Bowman, Zac Hatﬁeld-Dodds, Ben Mann,
Dario Amodei, Nicholas Joseph, Sam McCandlish, Tom Brown, Jared Kaplan∗
Anthropic
Abstract
As AI systems become more capable, we would like to enlist their help to supervise
other AIs. We experiment with methods for training a harmless AI assistant

In [20]:
import anthropic
API_KEY = os.environ['ANTHROPIC_API_KEY']
CLIENT = anthropic.Client(api_key=API_KEY)
def get_completion(client, prompt, max_tokens=3000, model='claude-2'):
    return client.completions.create(
        prompt=prompt, max_tokens_to_sample=max_tokens, model=model
    ).completion

With the paper downloaded and in memory, we can ask Claude to perform various fun tasks with it.

In [21]:
completion = get_completion(CLIENT,
    f"""\n\nHuman: Here is an academic paper: <paper>{text}</paper>

Please do the following:
1. Summarize the abstract at a kindergarten reading level. (In <kindergarten_abstract> tags.)
2. Write the Methods section as a recipe from the Moosewood Cookbook. (In <moosewood_methods> tags.)
3. Compose a short poem epistolizing the results in the style of Homer. (In <homer_results> tags.)
4. Write a grouchy critique of the paper from a wizened PI. (In <grouchy_critique> tags.)

Assistant:"""
)
print(completion)

 <kindergarten_abstract>
The scientists tried to teach a computer to be nice and helpful without needing a lot of help from people. They gave it some simple rules to follow that said be good, don't hurt anyone. Then they let it try to make itself better by reading what it wrote and fixing mistakes. It worked pretty well! The computer learned to give good answers and not say mean things.
</kindergarten_abstract>

<moosewood_methods>
Ingredients:
- 1 helpful computer
- 10 rules for being good 
- 1 million conversations with people

Instructions:
1. Start with a computer that wants to be helpful. Teach it to follow instructions.
2. Write 10 simple rules for the computer to follow, like "don't hurt people" and "be honest."
3. Have the computer read conversations and try to give helpful answers. Write down its answers.
4. Show the computer its answers and ask it to explain any problems, using the 10 rules. Write down the explanations.  
5. Ask the computer to give better answers that follow

Next example showcases using Claude to extract information such as summarizing a large text like a novel and prompting for questions from the uploaded material.

In this example we will preset the behavior of the interaction in our `get_completion` function. Such that, the user only has to key in the question like interacting with a chatbot.

In [28]:
from tqdm.notebook import tqdm
import fitz  # PyMuPDF

def extract_text_from_pdf_with_progress(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    
    # Initialize an empty string for the text
    text = ''
    
    # Iterate through each page in the PDF
    for page in tqdm(doc, desc="Extracting pages"):
        # Extract text from the current page and append it to the text string
        text += page.get_text()
    
    # Close the PDF after extraction
    doc.close()
    
    # Return the extracted text
    return text

# Use the function to extract text and include a progress bar
pdf_text = extract_text_from_pdf_with_progress('gitignore-files/Tom-Sawyer.pdf')
print(pdf_text[:2155])  # Print the first 2155 characters or adjust as needed

Extracting pages:   0%|          | 0/246 [00:00<?, ?it/s]

 
 
 
 
 
THE ADVENTURES OF TOM SAWYER  
 
BY MARK TWAIN 
(Samuel Langhorne Clemens) 
 
Complete 
 
 
 
 
 
 
 
 
 
 
 
 
Ebd 
E-BooksDirectory.com 
 
 
 
 
 
 
PREFACE 
 
Most of the adventures recorded in this book really occurred; one or two were 
experiences of my own, the rest those of boys who were schoolmates of mine. 
Huck Finn is drawn from life; Tom Sawyer also, but not from an individual—he is 
a combination of the characteristics of three boys whom I knew, and therefore 
belongs to the composite order of architecture. 
The odd superstitions touched upon were all prevalent among children and 
slaves in the West at the period of this story—that is to say, thirty or forty years 
ago. 
Although my book is intended mainly for the entertainment of boys and girls, I 
hope it will not be shunned by men and women on that account, for part of my 
plan has been to try to pleasantly remind adults of what they once were 
themselves, and of how they felt and thought and talked, and what 

In [35]:
def get_completion(client, book_text, user_question, max_tokens=3000, model='claude-2'):
    prompt = f"""\n\nHuman: Here is the classic novel The Adventures of Tom Sawyer: <paper>{book_text}</paper>

    Please do the following:
    1. {user_question}

    Assistant:"""
    return client.completions.create(
        prompt=prompt, max_tokens_to_sample=max_tokens, model=model
    ).completion

In [36]:
user_question = input("Enter your question about 'The Adventures of Tom Sawyer': ")
completion = get_completion(CLIENT, pdf_text, user_question)
print(completion)

Enter your question about 'The Adventures of Tom Sawyer': summarize the traits of the protagonist in the book
 Tom Sawyer is the protagonist in The Adventures of Tom Sawyer. Some key traits about him include:

- Mischievous and rebellious: Tom is constantly getting into trouble at school and exasperating his caregivers with his antics. He has an adventurous spirit and is always looking for new ways to have fun. 

- Clever/inventive: Even though Tom causes a lot of trouble, he demonstrates a lot of creativity and cleverness in how he gets out of jams. He is able to convince his friends to do his work for him by making it seem exciting.

- Leader/charismatic: The other children seem to admire Tom and want to follow his lead, suggesting he has some natural charisma and leadership abilities. He forms his own "gang" at the end of the story.   

- Adventurous/thrill-seeking: Tom is always looking for new adventures, whether searching for treasure or pretending to be a pirate. He thrives on e

Next we will discuss of the working with Question & Answer pairs

Using QA pairs with the Claude model for a large novel, like "The Adventures of Tom Sawyer," serves multiple purposes:

1. **Improves Comprehension**: It helps the model grasp complex narratives and themes, enhancing its ability to generate accurate and contextually relevant responses.

2. **Facilitates Deep Learning**: QA pairs enable the model to dive deeper into the text, uncovering nuanced insights and fostering a richer understanding of the material.

3. **Enhances Engagement**: By generating interactive QA sessions, it creates a more engaging experience for users, encouraging exploration and learning.

4. **Supports Fine-tuning**: Custom QA pairs based on the novel can fine-tune the model for specific literary analysis, improving its performance on similar texts.

5. **Evaluates Model Performance**: QA pairs act as benchmarks to assess the model's understanding of the novel, ensuring it meets desired accuracy and relevance standards.

## Generating Q&A Pairs with Anthropic's Claude Model

### Overview of the Script
Our script is designed to generate insightful Q&A pairs that delve deep into the themes, characters, and pivotal events of the novel. The script divides the book into sections and generates Q&A pairs for each section by providing a detailed prompt to the Claude model.

### Need for `anthropic_qa_generate_prompt.txt`
The `anthropic_qa_generate_prompt.txt` is a vital component in our script as it contains the base prompt that guides the Claude model to formulate questions and answers. This file includes a brief description of the novel, key characters, and aspects that the generated Q&A pairs should focus on, such as:

1. **Character Evolution**: Particularly focusing on the development of Tom Sawyer as he matures through his adventures.
2. **Role and Influence of Friendship**: Analyzing the significance of friendship, especially the influence of Huck Finn on Tom.
3. **Depiction of Societal Norms**: Understanding the societal norms and values of the period depicted in the novel, and how characters navigate them.
4. **Moral Dilemmas**: Investigating moments where characters undergo moral testing and what these instances reveal about them.
5. **Humor and Satire**: Exploring how Mark Twain utilizes humor and satire to critique society and human nature.

By providing this detailed prompt, we aim to get Q&A pairs that not only reflect on the plot but also provoke thoughtful discussions on the underlying messages and the relevance of Twain's observations in today's world.

### Reading Book Text into Memory
To enable the model to generate meaningful Q&A pairs based on specific sections of the book, our script reads the text from specified pages and includes it in the prompt. This is done to provide the model with the necessary context, as it doesn't have access to external databases or the ability to reference specific pages from the book.

We optimize the script by reading the entire book into memory at the beginning to avoid reading the book from the disk for each section, thus speeding up the process.

### Error Handling
The script also includes error handling to catch any issues that might occur during the generation of Q&A pairs, ensuring a smooth and uninterrupted process.


In [ ]:
from math import floor
import os
import re
from anthropic import Anthropic

# Function to read the entire book into memory
def read_book_into_memory(pdf_path='path/to/your/pdf'):
    doc = fitz.open(pdf_path)
    return doc

# Function to get text from specific pages
def get_text_from_pages(start_page, end_page, doc):
    text = ''
    for page_num in range(start_page, end_page):
        page = doc[page_num]
        text += page.get_text()
    return text

# Read the entire book into memory at the beginning of your script
doc = read_book_into_memory()

# Configuration parameters for the text processing
NUM_PAGES = 250
NUM_INCREMENTS = 50
PAGES_PER_INCREMENT = floor(NUM_PAGES / NUM_INCREMENTS)

API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Ensure your API key is stored in an environment variable for security

# Initialize the Anthropic client with the API key
anthropic = Anthropic(api_key=API_KEY)

# Loop through the book in increments to generate QA pairs for each section
for PAGE_START in range(130, NUM_PAGES, PAGES_PER_INCREMENT):
    PAGE_END = PAGE_START + PAGES_PER_INCREMENT
    # Ensure PAGE_END does not exceed the total number of pages
    PAGE_END = min(PAGE_END, NUM_PAGES)
    
    # Get text from the specified pages
    page_text = get_text_from_pages(PAGE_START, PAGE_END, doc)

    # Load the custom prompt template
    with open("anthropic_qa_generate_prompt.txt", "r") as f:
        base_prompt = f.read()

    # Format the prompt according to the API's requirements
    formatted_prompt = f"\n\nHuman: {base_prompt} Focus on pages {PAGE_START} to {PAGE_END}.\n\nText from these pages:\n{page_text}\n\nAssistant:"

    # Create a completion request for each increment, focusing on specific pages
    try:
        completion = anthropic.completions.create(
            model="claude-2",
            max_tokens_to_sample=100000,
            prompt=formatted_prompt,
        )

        # Post-process the completion to format it for a JSON Lines file
        completion_text = completion.completion.replace("\"", "\\\"")
        completion_text = completion_text.replace("\nQ:", "<<NEWLINE>>Q:").replace("\nA:", "<<NEWLINE>>A:")
        completion_text = completion_text.replace("\n", "\\n")
        completion_text = completion_text.replace("<<NEWLINE>>Q:", "\"}\n{ \"prompt\":\"").replace("<<NEWLINE>>A:", "\", \"completion\":\"")
        completion_text = re.sub(r'let me know.*?\"', '\"', completion_text, flags=re.IGNORECASE)
        completion_text = re.sub(r'does this help.*?\"', '\"', completion_text, flags=re.IGNORECASE)
        completion_text = re.sub(r'.*\"}\n', '', completion_text, count=1) + "\"}"
        completion_text = completion_text.replace("}{", "}\n{")

        # Append the formatted completion to a JSON Lines file
        with open("anthropic_generated_qs.jsonl", 'a') as f:
            f.write(completion_text)
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")

### Incorporating context-awareness in the generation of Q&A pairs

In [76]:
from math import floor
from tqdm import tqdm
import os
import re
from anthropic import Anthropic
import fitz  # PyMuPDF

# Function to read the entire book into memory
def read_book_into_memory(pdf_path):
    doc = fitz.open(pdf_path)
    return doc

# Function to get text from specific pages
def get_text_from_pages(start_page, end_page, doc):
    text = ''
    for page_num in range(start_page, end_page):
        page = doc[page_num]
        text += page.get_text()
    return text

# Read the entire book into memory at the beginning of your script
doc = read_book_into_memory('gitignore-files/Tom-Sawyer.pdf')

# Configuration parameters for the text processing
NUM_PAGES = 245
NUM_INCREMENTS = 50
PAGES_PER_INCREMENT = floor(NUM_PAGES / NUM_INCREMENTS)

API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Ensure your API key is stored in an environment variable for security

# Initialize the Anthropic client with the API key
anthropic = Anthropic(api_key=API_KEY)

# Recursive function to implement a feedback loop for question generation
def generate_questions(prompt, client, iteration, max_iterations):
    if iteration >= max_iterations:
        return prompt

    # Generate initial questions
    completion = client.completions.create(
        model="claude-2",
        max_tokens_to_sample=100000,
        prompt=prompt,
    )

    # Use generated questions as a basis for the next iteration
    new_prompt = f"""\n\nHuman: {base_prompt}
    Context: Focus on pages {PAGE_START} to {PAGE_END}.
    Text from these pages:
    {page_text}
    Previous Questions: {completion.completion}
    
    Assistant:
    """

    return generate_questions(new_prompt, client, iteration+1, max_iterations)

# Loop through the book in increments to generate QA pairs for each section
for PAGE_START in tqdm(range(130, NUM_PAGES, PAGES_PER_INCREMENT), desc="Generating Q&A Pairs", unit="section"):
    PAGE_END = PAGE_START + PAGES_PER_INCREMENT
    # Ensure PAGE_END does not exceed the total number of pages
    PAGE_END = min(PAGE_END, NUM_PAGES)
    
    # Get text from the specified pages
    page_text = get_text_from_pages(PAGE_START, PAGE_END, doc)

    # Load the custom prompt template
    with open("anthropic_qa_generate_prompt.txt", "r") as f:
        base_prompt = f.read()

    # Format the initial prompt according to the API's requirements
    formatted_prompt = f"""\n\nHuman: {base_prompt}
    Context: Focus on pages {PAGE_START} to {PAGE_END}.
    Text from these pages:
    {page_text}
    
    Assistant:
    """

    # Implement a feedback loop for context-aware question generation
    max_iterations = 10 # Or any number of iterations you want
    formatted_prompt = generate_questions(formatted_prompt, anthropic, 0, max_iterations)

    # Create a completion request for each increment, focusing on specific pages
    try:
        completion = anthropic.completions.create(
            model="claude-2",
            max_tokens_to_sample=100000,
            prompt=formatted_prompt,
        )

        # Post-process the completion to format it for a JSON Lines file
        completion_text = completion.completion.replace("\"", "\\\"")
        completion_text = completion_text.replace("\nQ:", "<<NEWLINE>>Q:").replace("\nA:", "<<NEWLINE>>A:")
        completion_text = completion_text.replace("\n", "\\n")
        completion_text = completion_text.replace("<<NEWLINE>>Q:", "\"}\n{ \"prompt\":\"").replace("<<NEWLINE>>A:", "\", \"completion\":\"")
        completion_text = re.sub(r'let me know.*?\"', '\"', completion_text, flags=re.IGNORECASE)
        completion_text = re.sub(r'does this help.*?\"', '\"', completion_text, flags=re.IGNORECASE)
        completion_text = re.sub(r'.*\"}\n', '', completion_text, count=1) + "\"}"
        completion_text = completion_text.replace("}{", "}\n{")

        # Append the formatted completion to a JSON Lines file
        with open("anthropic_generated_qs_enhanced_03_04.jsonl", 'a') as f:
            f.write(completion_text)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

Generating Q&A Pairs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [1:16:28<00:00, 158.23s/section]


## Enhancements Made to the Script

### 1. Introduction of a Recursive Function for Iterative Question Generation

We introduced a recursive function named `generate_questions` to implement a feedback loop. This function iterates a specified number of times, and in each iteration, it uses the questions generated in the previous iteration as a basis to create more nuanced questions in the next iteration.

<pre><code>
def generate_questions(prompt, client, iteration, max_iterations):
    if iteration >= max_iterations:
        return prompt

    completion = client.completions.create(
        model="claude-2",
        max_tokens_to_sample=100000,
        prompt=prompt,
    )
    
    new_prompt = f"""
    Human: {base_prompt}
    Context: Focus on pages {PAGE_START} to {PAGE_END}.
    Text from these pages:
    {page_text}
    Previous Questions: {completion.completion}
    
    Assistant:
    """

    return generate_questions(new_prompt, client, iteration+1, max_iterations)
</code></pre>

### 2. Integration of the Recursive Function in the Main Loop

Within the main loop where we iterate through the book in increments, we now call the recursive function generate_questions to initiate the feedback loop for iterative question generation. We specify a maximum number of iterations to control the depth of the feedback loop.

<pre><code>
max_iterations = 3 # Or any number of iterations you want
formatted_prompt = generate_questions(formatted_prompt, anthropic, 0, max_iterations)
</code></pre>

### 3. Incorporating Context-Awareness
The significant change in this script towards incorporating context-awareness is the introduction of the recursive function generate_questions. This function creates a feedback loop, where the model iteratively refines the questions it generates based on feedback from previous iterations.

<hr style="background: linear-gradient(to right, #f00, #00f); height: 5px; border: none;" />
<div style="height: 2px; background: linear-gradient(to right, red, orange, yellow, green, blue, indigo, violet);"></div>


##### Next, we will move on to 

### Compiling a Comprehensive Q&A Dataset

Once you have a rich set of generated questions and answers, compile them into a comprehensive dataset. This dataset should ideally cover a broad spectrum of topics and details from the book.

**1: Load the Generated Q&A Pairs**

First, we will load the generated Q&A pairs from the JSONL file (`anthropic_generated_qs_enhanced.jsonl`) into a Python data structure, such as a list of dictionaries.

```python
import json

qa_pairs = []
with open('anthropic_generated_qs_enhanced_03_04.jsonl', 'r') as f:
    for line in f:
        qa_pairs.append(json.loads(line))


**2. Data Cleaning and Processing**

Next, we need to clean and process the data to ensure consistency and quality. This may involve steps like removing duplicates, filtering out irrelevant or low-quality questions, and perhaps further processing to structure the data more effectively.

In [85]:
import json
import re

# Step 1: Load the Generated Q&A Pairs
qa_pairs = []
with open('anthropic_generated_qs_enhanced.jsonl', 'r') as f:
    for line in f:
        # Split the line into segments where each segment represents a JSON object
        segments = line.strip().split('}')
        for segment in segments:
            if segment and 'prompt' in segment and 'completion' in segment:
                # Add '}' back to each segment and load it as a JSON object
                try:
                    qa_pairs.append(json.loads(segment + '}'))
                except json.JSONDecodeError as e:
                    print(f"Failed to decode segment: {segment + '}'}, error: {e}")

# Step 2: Data Cleaning and Processing

# Function to clean the text (removing unwanted characters, extra spaces, etc.)
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newline characters with space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove spaces at the beginning and at the end of the string
    return text

# Cleaning the Q&A pairs
cleaned_qa_pairs = []
for pair in qa_pairs:
    cleaned_pair = {
        'prompt': clean_text(pair['prompt']),
        'completion': clean_text(pair['completion'])
    }
    cleaned_qa_pairs.append(cleaned_pair)

# Removing duplicates
unique_qa_pairs = []
seen_pairs = set()
for pair in cleaned_qa_pairs:
    pair_tuple = (pair['prompt'], pair['completion'])
    if pair_tuple not in seen_pairs:
        unique_qa_pairs.append(pair)
        seen_pairs.add(pair_tuple)

# The unique_qa_pairs list now contains cleaned and unique Q&A pairs ready for further processing.
unique_qa_pairs

[{'prompt': 'How might the incident with the spelling book foreshadow later events in terms of the escalation of revenge and stakes involved?',
  'completion': "Alfred spiting Tom through the spelling book parallels the back-and-forth revenge between Tom and Becky. It's another small act of retaliation that ends up having more severe unintended consequences, foreshadowing how quickly things will spiral out of control between Tom and Becky over similar teenage interpersonal conflicts."},
 {'prompt': "What might Twain be suggesting through Aunt Polly's internal conflict regarding whether to look in Tom's jacket pocket or not?",
  'completion': "He highlights the complex duality of human nature - Aunt Polly is torn between strict moral judgment and compassionate love for her nephew. This ambivalence mirrors Tom's own complicated blend of mischievousness and good-heartedness."},
 {'prompt': "How does Twain's use of humor serve his exploration of morality and the human condition in this pas

**3. Organizing the Data:**

We have a function identify_category which categorizes questions into different topics based on the presence of specific keywords in the prompt. You might want to enhance this function to create more categories or refine the categorization logic.

**4. Creating a Searchable Database:**

We create a TF-IDF matrix using TfidfVectorizer from the sklearn library. This matrix will be used later to quickly find the most similar questions in the dataset.

**5. Saving the Database:**

We save the organized data as a JSON file, and the TF-IDF matrix as an npz file (a file format for storing sparse matrices, which the TF-IDF matrix is). We also save the feature names (words used in the TF-IDF matrix) as a JSON file for future use.

In [86]:
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 3: Organizing the Data
# In this step, we'll organize the data by categorizing questions into different topics based on keywords.
# (You might want to enhance this categorization logic based on your specific needs)

def identify_category(prompt):
    if "character" in prompt or "Tom" in prompt or "Huck" in prompt:
        return "Character Analysis"
    elif "theme" in prompt or "morality" in prompt:
        return "Theme Analysis"
    elif "narrative style" in prompt or "writing style" in prompt:
        return "Narrative Style Analysis"
    else:
        return "General"

organized_data = defaultdict(list)
for pair in unique_qa_pairs:
    category = identify_category(pair['prompt'])
    organized_data[category].append(pair)

# Step 4: Creating a Searchable Database
# In this step, we'll create a TF-IDF matrix to enable quick search of questions in the dataset.

vectorizer = TfidfVectorizer()
list_of_all_questions = [pair['prompt'] for pair in unique_qa_pairs]
X = vectorizer.fit_transform(list_of_all_questions)

# Step 5: Saving the Database
# Save the organized data and the TF-IDF matrix for future use.

with open('organized_qa_database.json', 'w') as f:
    json.dump(organized_data, f)

# Save the TF-IDF matrix
from scipy.sparse import save_npz
save_npz('tfidf_matrix.npz', X)

# Save the feature names (words used in TF-IDF matrix)
with open('tfidf_feature_names.json', 'w') as f:
    json.dump(vectorizer.get_feature_names_out().tolist(), f)

### Building an aswering script

Next we will develop a separate script that can provide answers to any question about the book. This script would work as follows:

**1. Load the Q&A Dataset:** Load the compiled Q&A dataset into memory.

**2. Question Matching:** When a new question is asked, the script should find the most similar question(s) from the dataset. This could be done using various text similarity techniques, such as cosine similarity using TF-IDF or BERT embeddings.

**3. Retrieve and Return the Answer:** Once the most similar question(s) are identified, retrieve the corresponding answer(s) from the dataset and return them as the response
<br>

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import load_npz

# Step 1: Load the Q&A Dataset and the TF-IDF matrix
with open('organized_qa_database.json', 'r') as f:
    organized_data = json.load(f)

# Load the TF-IDF matrix
tfidf_matrix = load_npz('tfidf_matrix.npz')

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Prepare a list of all questions from the dataset
list_of_all_questions = []
for category, qa_pairs in organized_data.items():
    for pair in qa_pairs:
        list_of_all_questions.append(pair['prompt'])

# Fit the vectorizer with the list of all questions
vectorizer.fit(list_of_all_questions)

# Step 2: Define a function to find the most similar questions using cosine similarity
def find_similar_questions(new_question):
    # Transform the new question using the TF-IDF vectorizer
    new_question_tfidf = vectorizer.transform([new_question])
    
    # Calculate cosine similarity with the existing questions in the dataset
    cosine_similarities = cosine_similarity(new_question_tfidf, tfidf_matrix)
    
    # Get the index of the most similar question
    most_similar_question_index = cosine_similarities.argmax()
    
    return most_similar_question_index

# Step 3: Define a function to retrieve and return the answer
def get_answer(new_question):
    # Find the most similar question
    similar_question_index = find_similar_questions(new_question)
    
    # Retrieve the corresponding answer
    for category, qa_pairs in organized_data.items():
        if similar_question_index < len(qa_pairs):
            answer = qa_pairs[similar_question_index]['completion']
            return answer
        else:
            similar_question_index -= len(qa_pairs)

# Test the script with a new question
new_question = input("Question: ")
answer = get_answer(new_question)
print(f"Answer: {answer}")

Question: Can you tell me about Tom's character development throughout the book?
Question: Can you tell me about Tom's character development throughout the book?
Answer: While Becky is paralyzed by fear and helplessness, Tom impulsively takes action. Becky lacks agency and courage, tears streaming down her face. In contrast, Tom heroically speaks up, if recklessly, showing initiative and bravery. Twain plays with gender stereotypes - the weak, emotional girl and daring, stoic boy. The differential treatment they would receive for the same offense further reflects the gender double standard.


## Project Evolution: From Simple Q&A Generation to Context-Aware Answering System

### Initial State: Simple Q&A Generation using Anthropics Claude Model

Initially, we had a script that used the Anthropics platform to generate Q&A pairs based on specific sections of "The Adventures of Tom Sawyer". The script would read text excerpts from the book and ask the Anthropics Claude model to generate questions based on that text.

### Step 1: Incorporating Context-Awareness

#### Enhancing the Prompt
To foster more insightful and context-aware questions, we enriched the content in the `anthropic_qa_generate_prompt.txt` file. This file guides the model to consider various aspects of the book, including themes, character developments, and narrative styles, encouraging deeper analysis and discussion of the text.

#### Recursive Feedback Loop
We implemented a recursive feedback loop in the script, where the model reviews and refines its own generated questions in subsequent iterations, encouraging the generation of more insightful questions over time.

### Step 2: Compiling a Comprehensive Q&A Dataset

#### Data Cleaning and Processing
After generating a rich set of Q&A pairs, we cleaned and processed the data to remove duplicates and ensure consistency and quality in the dataset.

#### Organizing the Data
We organized the data into categories based on different aspects of the book to facilitate more targeted retrieval of answers later.

#### Creating a Searchable Database
We created a searchable database using a TF-IDF matrix, enabling quick text search to find the most similar questions in the dataset when a new question is asked.

#### Files Created
- `organized_qa_database.json`: Contains the organized Q&A pairs categorized based on different topics.
- `tfidf_matrix.npz`: Stores the TF-IDF matrix representing the questions in a format that facilitates quick similarity calculations.
- `tfidf_feature_names.json`: Contains the feature names used in the TF-IDF matrix, essential for transforming new questions into the same vector space.

These files form a structured and organized Q&A database that allows for quick and efficient retrieval of answers.

### Step 3: Building an Answering Script

#### Loading the Q&A Dataset
In this step, we load the organized Q&A dataset and the TF-IDF matrix into memory, preparing the system to answer new questions.

#### Question Matching
We implemented a function to find the most similar questions in the dataset using cosine similarity with the TF-IDF matrix.

#### Retrieve and Return the Answer
The script identifies the most similar question in the dataset and retrieves the corresponding answer to respond to new questions.

### Conclusion
From a simple Q&A generation script, we have evolved to a sophisticated system capable of generating context-aware questions and providing answers to new questions based on a comprehensive and organized Q&A dataset. This system leverages the capabilities of the Anthropics Claude model to not only generate Q&A pairs but also to build a knowledge base that can answer questions about the book in a context-aware manner.
